<a href="https://colab.research.google.com/github/Winzen/mides-rascunho/blob/main/code/scraping/se/%5Bse%5Dextrair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Constantes

In [ ]:
ano = 2016 # Ano de coleta
csv_coleta_name = None # caminho para o CSV que tem os empenhos desejado
path_drive_input = None # Caminho para salvar os empenhos coletados e extrair
path_drive_csv = None # Para salvar a pasta com os CSVs dos empenhos e extrair ela



# Verificar IP

In [ ]:
import requests

my_country = requests.get("https://api.myip.com/")
my_country.json()

{'ip': '35.194.240.248', 'country': 'Taiwan', 'cc': 'TW'}

# Importação

In [ ]:
import requests
import os
import csv
import shutil
import glob
import concurrent.futures
import datetime as dt
import time
import re
import pandas as pd
from time import sleep
from more_itertools import batched
from bs4 import BeautifulSoup, element
from urllib3 import PoolManager, disable_warnings, exceptions, Timeout
from urllib.parse import urlencode
from zipfile import ZipFile

def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)


def make_list_chunks(values: list, chunk_size:int = 2000) -> list:
  resquet_in_chunks = [chunk for chunk in batched(values, chunk_size)]
  return resquet_in_chunks


def extrair_zip(path, path_output) -> None:
  # loading the temp.zip and creating a zip object
  with ZipFile(path, 'r') as zObject:

      # Extracting all the members of the zip
      # into a specific location.
      zObject.extractall(
          path=path_output)

def send_solo_drive() -> None:

  drive.mount('/content/drive')

  send_folder_drive(path_drive_input + "_auto",
                    "/content/input/")

  send_folder_drive(path_drive_csv + "_auto",
                    "/content/csvs")

# Extrair CSV

In [ ]:
extrair_zip(path=path_drive_csv + ".zip",
        path_output="/content/csvs")

In [ ]:
extrair_zip(path=path_drive_input + ".zip",
        path_output="/content/input")

### Extrair Auto-Save

In [ ]:
extrair_zip(path=path_drive_csv + "_auto" + '.zip',
        path_output="/content/csvs")

In [ ]:
extrair_zip(path=path_drive_input + "_auto" + '.zip',
        path_output="/content/input")

# Raspagem

In [ ]:
def form_data_pagamentos(soup: BeautifulSoup, input_pagamento= "")-> str|dict:

  parametros = {inp["name"]:inp.get("value") for inp in soup.select("input")}

  data = {
  "__VIEWSTATE": parametros["__VIEWSTATE"],
  "__VIEWSTATEGENERATOR": parametros["__VIEWSTATEGENERATOR"],
  "__SCROLLPOSITIONX": parametros["__SCROLLPOSITIONX"],
  "__SCROLLPOSITIONY": parametros["__SCROLLPOSITIONY"],
  "__EVENTTARGET": "",
  "__EVENTARGUMENT": "",
  "__VIEWSTATEENCRYPTED": "",
  "__EVENTVALIDATION": parametros["__EVENTVALIDATION"],
  f"{input_pagamento}.x": "6",
  f"{input_pagamento}.y": "10",
  }

  encoded_params = urlencode(data)
  return encoded_params


def save_html(row: pd.DataFrame, response: requests.Response) ->  pd.DataFrame|bool:

  try:

    html = response.text

    if "DADOS DO EMPENHO" in html:
      path_base = f"input/{row.ano}/{row.municipio}/{row.unidade}/"

      os.makedirs(path_base, exist_ok=True)

      with open(path_base + f"{row.numero_empenho}.html", "w") as form_html:
        form_html.write(html)

      return row

    return False

  except:

    return False


def extrair(row: pd.DataFrame) -> pd.DataFrame|bool:
  try:

    headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0",
      "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
      "Accept-Language": "pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3",
      "Content-Type": "application/x-www-form-urlencoded",
      "Connection": "keep-alive"
    }

    with requests.Session() as s:

      empenho_pagamento = s.get(row.link, headers=headers, verify=False)
      soup_pagamento = BeautifulSoup(empenho_pagamento.text, "html.parser")

      elements_inputs = soup_pagamento.select("td input")

      if  elements_inputs:

        if len(elements_inputs) > 20: # Limitar numero de pagamentos aceitaveis
          return False

        input_ids = [element.get("id").replace("_", "$") for element in elements_inputs]

        for input in input_ids: # Abrir todos os inputs de pagamento
          data_pagamento = form_data_pagamentos(soup_pagamento, input)
          empenho_pagamento = s.post(row.link, headers=headers, data=data_pagamento, verify=False)
          soup_pagamento = BeautifulSoup(empenho_pagamento.text, "html.parser")

      return save_html(row, empenho_pagamento)

  except:
    return False

In [ ]:
df = pd.read_csv(csv_coleta_name, dtype=str)

In [ ]:
para_coletar = df[df["coletado"] == "False"]

chunks = make_list_chunks(para_coletar.itertuples(), 60)
disable_warnings(exceptions.InsecureRequestWarning)

for n, rows in enumerate(chunks):

  print(f"{'-' * 30}\nIniciando extração: {n + 1}/{len(chunks)}")
  start_time = time.time()

  with concurrent.futures.ThreadPoolExecutor() as executor:
    tasks = [task for task in executor.map(extrair, rows)]
    tasks_done = [task.Index for task in tasks if task]

  df.loc[tasks_done, "coletado"] = "True"

  print(f"De {len(rows)} paginas, foram coletadas {len(tasks_done)} com sucesso")

  segundo_execucao = time.time() - start_time

  print(f"Duração da execução: {segundo_execucao / 60:.2f}\n{'-' * 30}")
  print(f"Intervalo\nRespeito de {segundo_execucao * 0.2: 0.2f} segudos ao servidor")

  sleep(segundo_execucao * 0.2)

  if (n + 1) % 40 == 0:
    print("Salvando..")
    df.to_csv(csv_coleta_name, index=False)
    send_solo_drive()
    print("Salvamento concluido.")

print("Salvando..")
df.to_csv(csv_coleta_name, index=False)
send_solo_drive()
print("Salvamento concluido.")

## Verificar arquivos

In [ ]:
path_csvs = "/content/input/**/**/**/*.html"
htmls = glob.glob(path_csvs)
htmls = list(dict.fromkeys(htmls))
len(htmls)

207445

In [ ]:
registrados = df[df["coletado"] == "True"].shape[0]
print(len(htmls) == registrados)
print(f"Valores nesse momento são coletado {len(htmls)} e registrados {registrados}")

False
Valores nesse momento são coletado 207445 e registrados 207680


# Mandar no drive

In [ ]:
drive.mount('/content/drive')

df.to_csv(csv_coleta_name, index=False)

send_folder_drive(path_drive_input,
                "/content/input/")

send_folder_drive(path_drive_csv,
                "/content/csvs")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
